Testing FIBSEM Microscope overall class
    - Testing Thermo Fisher instance of class
    

Importing Relevant Modules

In [ ]:
%load_ext autoreload
%autoreload 2

from fibsem import utils, acquire
import fibsem.alignment as fa
from fibsem.structures import BeamType, ImageSettings, GammaSettings, FibsemRectangle, FibsemImage
import matplotlib.pyplot as plt
from fibsem import FibsemMicroscope
import os
import logging
from fibsem.utils import current_timestamp, configure_logging
from typing import Path
from autoscript_sdb_microscope_client.structures import (
    AdornedImage,
    GrabFrameSettings,
    Rectangle,
    RunAutoCbSettings,
)

Importing settings from config

In [ ]:
settings = utils.load_settings_from_config()


In [ ]:
session_path: Path = None,
config_path: Path = None,
protocol_path: Path = None,
setup_logging: bool = True,

In [ ]:
session = f'{settings.protocol["name"]}_{current_timestamp()}'
if protocol_path is None:
    protocol_path = os.getcwd()

# configure paths
if session_path is None:
    session_path = os.path.join(os.path.dirname(protocol_path), session)
os.makedirs(session_path, exist_ok=True)

# configure logging
if setup_logging:
    configure_logging(session_path)


# image_setttings
settings.image.save_path = session_path

logging.info(f"Finished setup for session: {session}")

Initialising a microscope instance (Thermo Fisher Microscope)

In [ ]:
microscope = FibsemMicroscope.ThermoMicroscope()

Testing Connection 

In [ ]:
microscope.connect()

Setting up for image acquisition

In [ ]:
# set gamma settings
gamma_settings = GammaSettings(
        enabled=True,
        min_gamma=0.5,
        max_gamma=1.8,
        scale_factor=0.01,
        threshold=46,
    )

# set imaging settings
image_settings = ImageSettings(
        resolution="1536x1024",
        dwell_time=1.0e-6,
        hfw=150.0e-6,
        autocontrast=True,
        beam_type=BeamType.ELECTRON,
        gamma=gamma_settings,
        save=False,
        save_path="path",
        label="label",
        reduced_area=None,
    )

frame_settings = GrabFrameSettings(
    resolution=image_settings.resolution,
    dwell_time=image_settings.dwell_time,
    reduced_area=image_settings.reduced_area,
)

 # set label
if image_settings.beam_type is BeamType.ELECTRON:
    label = f"{settings.label}_eb"

if image_settings.beam_type is BeamType.ION:
    label = f"{settings.label}_ib"

Autoconcrast

In [ ]:
microscope.autocontrast(beam_type=image_settings.beam_type)

Take new image

In [ ]:
new_image = microscope.acquire_image(frame_settings, image_settings)
# apply gamma correction
if settings.gamma.enabled:
    image = acquire.auto_gamma(new_image, settings.gamma)

plt.imshow(new_image.data)
plt.show()

Take last image

In [ ]:
new_image = microscope.last_image(image_settings.beam_type.value)

plt.imshow(new_image.data)
plt.show()